In [1]:
import os, sys, time
import numpy as np

from BQ_code.utils import FUNC, ALGO
from BQ_code.functions import *
from BQ_code.BQ import GPBQ
from BQ_code.utils import calc_quadrature_trap, calc_quadrature_mc, calc_quadrature_bq

In [2]:
NUM_TRIAL = 100
NUM_EXP = 1
MAXSTEP = 250
N_INITS = 3

In [3]:

def run(base_dir, func_name, algorithms, noise_level, split=0.5, save_split=False):
    _result_path = os.path.join(base_dir, FUNC[func_name])
    if not os.path.exists(_result_path):
        os.mkdir(_result_path) 

    _result_path = os.path.join(_result_path, str(noise_level))
    if not os.path.exists(_result_path):
        os.mkdir(_result_path)

    if save_split:
        _result_path = os.path.join(_result_path, str(split))
        if not os.path.exists(_result_path):
            os.mkdir(_result_path)

    function = eval(FUNC[func_name])(noisy=True, noise_std=noise_level)
    func_bounds=function.bounds

    
    for j in range(NUM_TRIAL):
        '''
        In each trial, all the algorithms share the same initial points
        Different trials have different initial points
        '''
    
        X_init = np.random.uniform(func_bounds[:, 0], func_bounds[:, 1], size=(N_INITS, func_bounds.shape[0]))

        for algo_name in algorithms:
            result_path = os.path.join(_result_path, ALGO[algo_name])
            if not os.path.exists(result_path):
                os.mkdir(result_path)
            log_path = os.path.join(result_path, str(func_name) + '_' + str(algo_name) + '_log.log')
            log = open(log_path, 'a')
            sys.stdout = log

            history_save_path = os.path.join(result_path, str(func_name) + '_' + str(algo_name) + '_query_history.npy')
            if not os.path.exists(history_save_path):
                history = {'observedpoints' : [],
                        'observedvalues': [],
                        'integrals' : [],
                        }
            else:
                history = np.load(history_save_path, allow_pickle=True)[()]


            print(str(j+1) + 'th trial of ' + ALGO[algo_name])

            for i in range(NUM_EXP):
                '''
                Run each algorithm $NUM_EXP times
                '''

                start_time = time.time()
                print('\t' + str(i+1) + 'th experiment of ' + ALGO[algo_name])
                sys.stdout.flush()


                # hyper_history = []
                if algo_name in ['mvs-mc-mat', 'mvs-mat', 'mvs-se', 'mvs-mc-se']:
                    '''
                    ensure mvs-mc and mvs have same initial points
                    '''
                    Bq_test= GPBQ(function, func_bounds, algo_name)
                    Bq_test.set_hyper(0.2, 1, 1.5) #l, var, nu
                    Bq_test.gp.noise_delta = 1e-5

                    Bq_test.initiate(X_init)

                    if 'mvs-mc' in algo_name:
                        quad_his, x_his, y_his = calc_quadrature_bq(Bq_test, function, func_bounds, MAXSTEP, N_INITS, split=split)
                    else:
                        quad_his, x_his, y_his = calc_quadrature_bq(Bq_test, function, func_bounds, MAXSTEP, N_INITS, split=1)

                    del Bq_test

                elif algo_name == 'mc':
                    quad_his, x_his, y_his = calc_quadrature_mc(function, func_bounds, MAXSTEP)
                elif algo_name == 'trap':
                    quad_his, x_his, y_his = calc_quadrature_trap(function, func_bounds, MAXSTEP)


                history['observedpoints'].append(x_his)
                history['observedvalues'].append(y_his)
                history['integrals'].append(quad_his)
                sys.stdout.flush()

                current_time = time.time()
                print('\tRunning time : ', current_time - start_time)
                print('\n')

            np.save(history_save_path, np.asarray(history))

        del X_init


### Main experiment

In [4]:
NOISE_LEVEL = [0, 1e-1, 5e-1]
FUNCS = ['ack1']  # More supported functions can be found in ./BQ_code/utils.py

base_dir = './results'
if not os.path.exists(base_dir):
    os.mkdir(base_dir)

In [ ]:
for func_name in FUNCS:
    for noise in NOISE_LEVEL:
        run(base_dir, func_name, ['mc','mvs-se','mvs-mc-se','mvs-mat','mvs-mc-mat'], noise, 0.5)

### Split experiment

In [6]:
NUM_TRIAL = 10
SPLITS = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

base_dir = './results-split'
if not os.path.exists(base_dir):
    os.mkdir(base_dir)

In [10]:
for func_name in FUNCS:
    for noise in NOISE_LEVEL:
        for split in SPLITS:
            run(base_dir, func_name, ['mvs-mc-mat'], noise, split, True)